你的推理过程（我帮你理清一下）

目标变量

你最终关心的是 地图复杂度 Complexity

但 Complexity 不能直接观测（它只是一个潜在概念/指标）

我们能观测到的结果是 success_rate（训练表现）

核心假设

success_rate 越低 → 地图越难（Complexity 越高）

success_rate 越高 → 地图越简单（Complexity 越低）

所以：Complexity 是 success_rate 的函数。

学习流程

Step 1. 用 XGBoost 学习 success_rate ~ 特征(size, agents, density, LDD, BN, MC, DLR, ...) 的非线性关系

Step 2. 提取特征重要性（说明哪些特征最影响 success_rate → 哪些特征决定复杂度）

Step 3. 把 XGBoost 的非线性结果转化成线性权重公式

In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# 读数据
df = pd.read_csv("C:/Users/MSc_SEIoT_1/MAPF_G2RL-main/logs/train_gray3d-Copy.csv")

# 特征 & 目标
features = ["size", "num_agents", "density", "density_actual", "LDD", "BN", "MC", "DLR"]
X = df[features]
y = df["success_rate"]

# Step 1: 用 XGBoost 学 success_rate
xgb_model = xgb.XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(X, y)

# Step 2: 特征重要性
importance = pd.DataFrame({
    "feature": features,
    "importance": xgb_model.feature_importances_
}).sort_values("importance", ascending=False)

print("📊 特征重要性 (对 success_rate 的影响)：")
print(importance)

# Step 3: 转化成线性权重（拟合可解释公式）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lr = LinearRegression().fit(X_scaled, y)

weights = pd.DataFrame({
    "feature": features,
    "weight": lr.coef_
}).sort_values("weight", ascending=False)

print("\n📊 线性权重 (可解释公式)：")
print(weights)

# Step 4: Complexity = 1 - success_rate
# 你也可以直接用 1 - success_rate 或者把 success_rate 回归结果反转
df["Complexity"] = 1 - y

print("\n公式：")
print("Complexity = {:.3f} + ".format(lr.intercept_) +
      " + ".join(f"({w:.3f} * {f})" for f, w in zip(features, lr.coef_)))


📊 特征重要性 (对 success_rate 的影响)：
          feature  importance
2         density    0.367306
6              MC    0.200028
3  density_actual    0.128360
4             LDD    0.107434
5              BN    0.062293
0            size    0.046418
7             DLR    0.044426
1      num_agents    0.043736

📊 线性权重 (可解释公式)：
          feature    weight
3  density_actual  0.131938
2         density  0.077375
6              MC  0.039424
4             LDD  0.026888
0            size  0.021092
7             DLR  0.002133
1      num_agents -0.010119
5              BN -0.127655

公式：
Complexity = 0.848 + (0.021 * size) + (-0.010 * num_agents) + (0.077 * density) + (0.132 * density_actual) + (0.027 * LDD) + (-0.128 * BN) + (0.039 * MC) + (0.002 * DLR)


In [ ]:
# ================================
# XGBoost 特征学习 + 可视化 + 权重提取 + 线性公式
# ================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# ---------- 0) 参数 ----------
CSV_PATH = "C:/Users/MSc_SEIoT_1/MAPF_G2RL-main/logs/train_gray3d-Copy.csv"   # 改成你的路径
OUTDIR   = "C:/Users/MSc_SEIoT_1/MAPF_G2RL-main/pics1"                      # 图片和结果输出目录
os.makedirs(OUTDIR, exist_ok=True)

# 使用到的特征与目标
FEATURES = ["size", "num_agents", "density", "density_actual", "LDD", "BN", "MC", "DLR"]
TARGET   = "success_rate"  # 首选目标

# ---------- 1) 读数据 & 清洗 ----------
df = pd.read_csv(CSV_PATH, engine="python")

# 强制转数值
for col in FEATURES + [TARGET]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 选择目标：优先 success_rate；如果全无效，回退到 Complexity proxy
use_success = False
if TARGET in df.columns:
    y_ok = df[TARGET].notna() & np.isfinite(df[TARGET]) & (df[TARGET] >= 0) & (df[TARGET] <= 1)
    if int(y_ok.sum()) > 0:
        use_success = True
        y = df.loc[y_ok, TARGET]
        X = df.loc[y_ok, FEATURES]
    else:
        y = df[["LDD", "BN", "MC", "DLR"]].mean(axis=1)
        X = df[FEATURES]
else:
    y = df[["LDD", "BN", "MC", "DLR"]].mean(axis=1)
    X = df[FEATURES]

# 特征用中位数补缺，保证稳定绘图
X = X.fillna(X.median(numeric_only=True))

# ---------- 2) 划分数据 ----------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ---------- 3) 训练 XGBoost ----------
xgb_model = xgb.XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",
    random_state=42
)
xgb_model.fit(X_train, y_train)

# ---------- 4) 评估 ----------
y_pred_tr = xgb_model.predict(X_train)
y_pred_te = xgb_model.predict(X_test)

r2_tr = r2_score(y_train, y_pred_tr)
r2_te = r2_score(y_test, y_pred_te)
rmse_te = mean_squared_error(y_test, y_pred_te, squared=False)
mae_te = mean_absolute_error(y_test, y_pred_te)

print(f"Target = {'success_rate' if use_success else 'Complexity proxy (mean of LDD/BN/MC/DLR)'}")
print(f"R^2 train = {r2_tr:.3f}, R^2 test = {r2_te:.3f}, RMSE test = {rmse_te:.4f}, MAE test = {mae_te:.4f}")

# ---------- 5) 内置特征重要性 ----------
imp_df = pd.DataFrame({
    "feature": FEATURES,
    "importance": xgb_model.feature_importances_
}).sort_values("importance", ascending=True)

plt.figure(figsize=(7, 4.5))
plt.barh(imp_df["feature"], imp_df["importance"])
plt.xlabel("Importance")
plt.title("XGBoost Feature Importance" + (" (success_rate)" if use_success else " (Complexity proxy)"))
plt.tight_layout()
fi_path = os.path.join(OUTDIR, "xgb_feature_importance.png")
plt.savefig(fi_path)
plt.close()

# ---------- 6) Permutation Importance（测试集，更稳健） ----------
pi = permutation_importance(
    xgb_model, X_test, y_test, n_repeats=15, random_state=42, n_jobs=1
)
pi_df = pd.DataFrame({
    "feature": FEATURES,
    "perm_importance": pi.importances_mean,
    "perm_std": pi.importances_std
}).sort_values("perm_importance", ascending=True)

plt.figure(figsize=(7, 4.5))
plt.barh(pi_df["feature"], pi_df["perm_importance"])
plt.xlabel("Permutation Importance (mean)")
plt.title("Permutation Importance on Test Set")
plt.tight_layout()
pi_path = os.path.join(OUTDIR, "permutation_importance.png")
plt.savefig(pi_path)
plt.close()

# ---------- 7) 预测 vs 实际 ----------
xy_min = float(min(y_test.min(), y_pred_te.min()))
xy_max = float(max(y_test.max(), y_pred_te.max()))

plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred_te, s=16)
plt.plot([xy_min, xy_max], [xy_min, xy_max])
plt.xlabel("Actual")
plt.ylabel("Predicted")
ttl = "Predicted vs Actual (XGBoost)"
ttl += f"\nR²={r2_te:.3f}, RMSE={rmse_te:.3f}, MAE={mae_te:.3f}"
plt.title(ttl)
plt.tight_layout()
pva_path = os.path.join(OUTDIR, "pred_vs_actual.png")
plt.savefig(pva_path)
plt.close()

# ---------- 8) 残差图 ----------
residuals = y_test - y_pred_te
plt.figure(figsize=(7, 4))
plt.scatter(y_pred_te, residuals, s=12)
plt.axhline(0)
plt.xlabel("Predicted")
plt.ylabel("Residual (Actual - Pred)")
plt.title("Residuals vs Predicted")
plt.tight_layout()
rvp_path = os.path.join(OUTDIR, "residuals_vs_pred.png")
plt.savefig(rvp_path)
plt.close()

# 残差直方图
plt.figure(figsize=(7, 4))
plt.hist(residuals, bins=20)
plt.xlabel("Residual")
plt.ylabel("Count")
plt.title("Residuals Histogram")
plt.tight_layout()
rhist_path = os.path.join(OUTDIR, "residuals_hist.png")
plt.savefig(rhist_path)
plt.close()

# ---------- 9) 前 3 特征的 PDP ----------
top3 = list(imp_df.sort_values("importance", ascending=False)["feature"][:3])
baseline = X_train.median(numeric_only=True)
pdp_paths = []

for f in top3:
    f_min, f_max = X_train[f].min(), X_train[f].max()
    grid = np.linspace(f_min, f_max, 70)

    # 基线为各特征中位数，仅改变一个特征 f
    X_grid = np.repeat(baseline.values.reshape(1, -1), len(grid), axis=0)
    X_grid = pd.DataFrame(X_grid, columns=FEATURES)
    X_grid[f] = grid

    preds = xgb_model.predict(X_grid)

    plt.figure(figsize=(7, 4))
    plt.plot(grid, preds)
    plt.xlabel(f)
    plt.ylabel("Predicted " + ("success_rate" if use_success else "Complexity proxy"))
    plt.title("Partial Dependence: " + f)
    plt.tight_layout()
    outp = os.path.join(OUTDIR, f"pdp_{f}.png")
    plt.savefig(outp)
    plt.close()
    pdp_paths.append(outp)

# ---------- 10) 基于 Permutation Importance 的“Complexity 权重” ----------
# 若有负值（噪声造成的），裁剪为0后再归一化
pi_pos = pi_df.copy()
pi_pos["perm_importance_clipped"] = pi_pos["perm_importance"].clip(lower=0)
total = pi_pos["perm_importance_clipped"].sum()
if total == 0:
    # 如果全是0，退化为等权
    pi_pos["weight_norm"] = 1.0 / len(pi_pos)
else:
    pi_pos["weight_norm"] = pi_pos["perm_importance_clipped"] / total

# 为了直观展示“对复杂度的贡献”，我们通常让“成功率的负相关特征”获得正的复杂度权重。
# 这里简单做一个符号翻转：如果 XGBoost 的 SHAP 或方向没有估，先给出无方向的正权重（用在加权综合上）。
# 你也可以后续用 SHAP 判断方向，再给权重加 +/- 号。
weights_csv = os.path.join(OUTDIR, "complexity_weights_from_PI.csv")
pi_pos[["feature", "weight_norm"]].sort_values("weight_norm", ascending=False).to_csv(weights_csv, index=False)

print("\n=== Complexity 权重（基于 Permutation Importance 归一化） ===")
print(pi_pos[["feature", "weight_norm"]].sort_values("weight_norm", ascending=False))

# ---------- 11) 线性可解释公式（模型蒸馏） ----------
# 用 XGBoost 的预测作为 teacher target，对标准化特征做线性回归，得到可解释公式
scaler = StandardScaler()
X_tr_scaled = scaler.fit_transform(X_train)
X_te_scaled = scaler.transform(X_test)

teacher_tr = xgb_model.predict(X_train)
teacher_te = xgb_model.predict(X_test)

lin = LinearRegression().fit(X_tr_scaled, teacher_tr)
coef = lin.coef_
intercept = lin.intercept_

formula = "ŷ = {:.6f} + ".format(intercept) + " + ".join(
    f"({w:+.6f}·{name}_z)" for w, name in zip(coef, FEATURES)
)

print("\n=== 线性可解释“蒸馏”公式（输入为标准化后的 z 分数） ===")
print(formula)

# 保存线性系数
coef_df = pd.DataFrame({"feature": FEATURES, "coef_on_z": coef})
coef_path = os.path.join(OUTDIR, "linear_surrogate_coeffs.csv")
coef_df.sort_values("coef_on_z", ascending=False).to_csv(coef_path, index=False)

# ---------- 12) 路径汇总 ----------
print("\n=== 输出文件 ===")
print("Feature importance:", fi_path)
print("Permutation importance:", pi_path)
print("Pred vs Actual:", pva_path)
print("Residuals vs Pred:", rvp_path)
print("Residuals histogram:", rhist_path)
print("PDP (top 3):", ", ".join(pdp_paths))
print("Complexity 权重 CSV:", weights_csv)
print("线性系数 CSV:", coef_path)


Target = success_rate
R^2 train = 0.930, R^2 test = 0.713, RMSE test = 0.0557, MAE test = 0.0455

=== Complexity 权重（基于 Permutation Importance 归一化） ===
          feature  weight_norm
2         density     0.952769
3  density_actual     0.019045
4             LDD     0.011365
6              MC     0.010588
0            size     0.006232
7             DLR     0.000000
1      num_agents     0.000000
5              BN     0.000000

=== 线性可解释“蒸馏”公式（输入为标准化后的 z 分数） ===
ŷ = 0.849477 + (+0.030566·size_z) + (-0.005275·num_agents_z) + (+0.076298·density_z) + (+0.179387·density_actual_z) + (+0.030260·LDD_z) + (-0.177219·BN_z) + (+0.042741·MC_z) + (+0.000663·DLR_z)

=== 输出文件 ===
Feature importance: C:/Users/MSc_SEIoT_1/MAPF_G2RL-main/pics\xgb_feature_importance.png
Permutation importance: C:/Users/MSc_SEIoT_1/MAPF_G2RL-main/pics\permutation_importance.png
Pred vs Actual: C:/Users/MSc_SEIoT_1/MAPF_G2RL-main/pics\pred_vs_actual.png
Residuals vs Pred: C:/Users/MSc_SEIoT_1/MAPF_G2RL-main/pics\residuals_